In [2]:
from utils import time_op
import impmeas as imp

In [3]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = imp.formulas.GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    satcount_bdd = time_op(f.expectation)() * 2**n
    print(f"BDD satcount = {satcount_bdd}")
    del f

[00007.0219 ms / 0.0070 s / 0.0001 min] satcount
GPMC satcount = 223
[00072.1278 ms / 0.0721 s / 0.0012 min] parse
[00000.0062 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 223.0


In [4]:
n,clauses_to_variables,k = 15, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [5]:
import impmeas.bdds as bdd 

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    x = "x1"
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")
    del f

[00983.3860 ms / 0.9834 s / 0.0164 min] parse
bdd size = 856
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f9211499c00> ===
k=0 d result=0.2319 max increase possible=0.3840 
k=1 d result=0.2067 max increase possible=0.1182 
k=2 d result=0.1043 max increase possible=0.0105 
k=3 d result=0.0103 max increase possible=0.0001 
k=4 d result=0.0001 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5533, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[10905.8199 ms / 10.9058 s / 0.1818 min] blame
[00000.0699 ms / 0.0001 s / 0.0000 min] influence
influence 0.23193, 0.55332 <= blame


In [6]:
solver = imp.formulas.GPMC(cs=20000)
imp.formulas.set_pmc_solver(solver)
f = time_op(imp.Formula.parse)(formula)
x = "x1"
infl = (time_op)(imp.influence)(f, x)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[01081.4509 ms / 1.0815 s / 0.0180 min] parse
[00212.5692 ms / 0.2126 s / 0.0035 min] influence
=== COMPUTING BLAME for x1 in Formula with size 2774 ===
k=0 size of cnf: 3079 d result=0.2319 max increase possible=0.3840 
k=1 size of cnf: 3130 d result=0.2067 max increase possible=0.1182 
k=2 size of cnf: 3177 d result=0.1043 max increase possible=0.0105 
k=3 size of cnf: 3220 d result=0.0103 max increase possible=0.0001 
k=4 size of cnf: 3259 d result=0.0001 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.5533, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[05531.7063 ms / 5.5317 s / 0.0922 min] blame
influence 0.23193, 0.55332 <= blame


In [9]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)

with imp.formulas.BuddyContext(vars) as ctx:
    imp.formulas.set_buddy_context(ctx)
    f = time_op(imp.BuddyNode.parse)(formula)
    print(f"bdd size = {f.nodecount}")
    blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(imp.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00924.0031 ms / 0.9240 s / 0.0154 min] parse
[01155.3473 ms / 1.1553 s / 0.0193 min] blame
0.65894 <= blame
[00013.2289 ms / 0.0132 s / 0.0002 min] influence
influence = 0.38281
[06540.7891 ms / 6.5408 s / 0.1090 min] <lambda>
hammer = 0.11516
[00681.9389 ms / 0.6819 s / 0.0114 min] dominating_cgm
[00558.7838 ms / 0.5588 s / 0.0093 min] rectifying_cgm
[00537.9312 ms / 0.5379 s / 0.0090 min] parse
bdd size = 96
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f91f29501f0> ===
k=0 d result=0.3828 max increase possible=0.3086 
k=1 d result=0.2129 max increase possible=0.0638 
k=2 d result=0.0615 max increase possible=0.0017 
k=3 d result=0.0017 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.6589, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00197.4239 ms / 0.1974 s / 0.0033 min] blame
[00000.0386 ms / 0.0000 s / 0.0000 min] influence
influence 0.38281, 0.65894 <= blame
